In [1]:
import pandas as pd
import os
import datetime

In [2]:
list_data_directories = os.listdir('../data_pmd')
print(list_data_directories)

['FY19', 'FY20', 'FY21', 'FY22']


## Prepare Data - FY20

In [3]:
columns_fy20 = ['Soff', 'Sold to Party', 'Period', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm', 'FY', 'Total New Ord/GG1101', 'Revenue/GG1201', 'Sales Margin/GG1401', 'REV Qty']
dtypes_fy20 = {'Soff': 'str', 'Sold to Party': 'str', 'Period': 'str', 'Prctr': 'str', 'WBS Element': 'str', 'PCK /FGK': 'str', 'Matnr Sitm': 'str', 'FY': 'str', 'Total New Ord/GG1101': 'float64', 'Revenue/GG1201': 'float64', 'Sales Margin/GG1401': 'float64', 'REV Qty': 'int64'}

# Read data in directory FY20 and store it in a dataframe with pandas
files = os.listdir('../data_pmd/FY20')
df_list = []
for file in files:
    df = pd.read_csv(f'../data_pmd/FY20/{file}', usecols=columns_fy20, dtype=dtypes_fy20)
    df_list.append(df)

# Merge all dataframes into one dataframe for FY20
df_fy20 = pd.concat(df_list)
df_fy20.shape


(747042, 12)

In [4]:
# Fill missing values with '(en blanco)'
df_fy20[['Soff', 'Sold to Party', 'Period', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm', 'FY']] = df_fy20[['Soff', 'Sold to Party', 'Period', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm', 'FY']].fillna('(en blanco)')

In [5]:
# group by 'Soff', 'Sold to Party', 'Period', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm' and 'FY' and aggregate sum of 'Total New Ord/GG1101', 
# 'Revenue/GG1201', 'Sales Margin/GG1401' and 'REV Qty'
df_fy20_grouped = df_fy20.groupby(['Soff', 'Sold to Party', 'Period', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm', 'FY'], as_index=False) \
    .agg({'Total New Ord/GG1101': 'sum', 'Revenue/GG1201': 'sum', 'Sales Margin/GG1401': 'sum', 'REV Qty': 'sum'})
df_fy20_grouped.shape

(333075, 12)

## Prepare Data - FY21

In [6]:
columns_fy21 = ['Soff', 'Sold to Party', 'Period', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm', 'FY','Sdoc Nr', 'GBK/GCK', 'Total New Ord/GG1101', 'Revenue/GG1201', 'Sales Margin/GG1401', 'REV Qty']
dtypes_fy21 = {'Soff': 'str', 'Sold to Party': 'str', 'Period': 'str', 'Prctr': 'str', 'WBS Element': 'str', 'PCK /FGK': 'str', 'Matnr Sitm': 'str', 'FY': 'str', 'Sdoc Nr': 'str', 'GBK/GCK': 'str', 'Total New Ord/GG1101': 'float64', 'Revenue/GG1201': 'float64', 'Sales Margin/GG1401': 'float64', 'REV Qty': 'int64'}

# Read data in directory FY21 and store it in a dataframe with pandas
files = os.listdir('../data_pmd/FY21')
df_list = []
for file in files:
    df = pd.read_csv(f'../data_pmd/FY21/{file}', usecols=columns_fy21, dtype=dtypes_fy21)
    df_list.append(df)

# Merge all dataframes into one dataframe for FY21
df_fy21 = pd.concat(df_list)
df_fy21.shape

(699712, 14)

Corregir error de SAP para documentos de venta en periodo 2

In [7]:
# Read 'Sdoc Nr' from 'sales_documents_p1p3' document and convert in string de data
df_sales_documents_p1p3 = pd.read_excel('../bases/sales_documents_p1p3.xlsx', usecols=['Sdoc Nr'], dtype={'Sdoc Nr': 'str'})

In [8]:
# Replace 'Prctr' in df_fy21 with value 'P10300200'. Where columns 'Period' = 'P02' and 'Sdoc Nr' in list_sdoc_nr
df_fy21.loc[df_fy21['Sdoc Nr'].isin(df_sales_documents_p1p3['Sdoc Nr']) & (df_fy21['Period'] == 'P02'), 'Prctr'] = 'P10300200'
df_fy21.shape

(699712, 14)

In [9]:
# Add a column 'Tipo' with values 'Canales' to df_fy21 where column 'GBK/PCK' = R711 and R750
df_fy21.loc[df_fy21['GBK/GCK'].isin(['R711', 'R750']), 'Tipo'] = 'Canales'


In [10]:
# Fill empty values with '(en blanco)'
df_fy21[['Soff', 'Sold to Party', 'Period', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm', 'FY']] = df_fy21[['Soff', 'Sold to Party', 'Period', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm', 'FY']].fillna('(en blanco)')

In [11]:
# group by 'Soff', 'Sold to Party', 'Period', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm' and 'FY' and aggregate sum of 'Total New Ord/GG1101', 
# 'Revenue/GG1201', 'Sales Margin/GG1401' and 'REV Qty' and aggregate first value of 'Tipo'
df_fy21_grouped = df_fy21.groupby(['Soff', 'Sold to Party', 'Period', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm', 'FY'], as_index=False) \
    .agg({'Total New Ord/GG1101': 'sum', 'Revenue/GG1201': 'sum', 'Sales Margin/GG1401': 'sum', 'REV Qty': 'sum', 'Tipo': 'first'})
df_fy21_grouped.shape

(320111, 13)

Clasificar tipo: "Canales" y "Proyectos" de cada línea

In [12]:
# Read data from 'sales_documents_p1p3' document
df_pcks_p4 = pd.read_excel('../bases/pcks_p4.xlsx', usecols=['GBK/PCK'], dtype={'GBK/PCK': 'str'})

In [13]:
# Replace values of 'Tipo' with 'Canales' in df_fy21_grouped where column 'PCK /FGK' in df_pcks_p4_list
df_fy21_grouped.loc[df_fy21_grouped['PCK /FGK'].isin(df_pcks_p4['GBK/PCK']), 'Tipo'] = 'Canales'

In [14]:
# Replace values of 'Tipo' with 'Proyectos' in df_fy21_grouped where column 'Sold to Party' = '41003426'
df_fy21_grouped.loc[df_fy21_grouped['Sold to Party'] == '41003426', 'Tipo'] = 'Proyectos'
df_fy21_grouped.shape

(320111, 13)

## Prepare Data - FY22

In [15]:
columns_fy22 = ['GBK/GCK', 'Period', 'Soff', 'Sold to Party', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm','Total New Ord/GG1101', 'Revenue/GG1201', 'Sales Margin/GG1401', 'REV Qty']
dtypes_fy22 = {'GBK/GCK': 'str', 'Period': 'str', 'Soff': 'str', 'Sold to Party': 'str', 'Prctr': 'str', 'WBS Element': 'str', 'PCK /FGK': 'str', 'Matnr Sitm': 'str', 'Total New Ord/GG1101': 'float64', 'Revenue/GG1201': 'float64', 'Sales Margin/GG1401': 'float64', 'REV Qty': 'float64'}

# Read data in directory FY21 and store it in a dataframe with pandas
files = os.listdir('../data_pmd/FY22')
df_list = []
for file in files:
    df = pd.read_csv(f'../data_pmd/FY22/{file}', usecols=columns_fy22, dtype=dtypes_fy22)
    df_list.append(df)

# Merge all dataframes into one dataframe for FY21
df_fy22 = pd.concat(df_list)
df_fy22.shape

(351505, 12)

In [16]:
# Add a column 'Tipo' with values 'Canales' to df_fy22 where column 'GBK/PCK' = R711 and R750
df_fy22.loc[df_fy22['GBK/GCK'].isin(['R711', 'R750']), 'Tipo'] = 'Canales'
df_fy22['FY'] = 'FY22'
df_fy22.shape

(351505, 14)

In [17]:
# Replace first character of 'Period' column values with 'P'
df_fy22['Period'] = df_fy22['Period'].str.replace('0', 'P', 1)

Preparar data, llenando celdas vacías para poder realizar la agrupación y agregación

In [18]:
# Fill empty values with '(en blanco)'
df_fy22[['Soff', 'Sold to Party', 'Period', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm', 'FY']] = df_fy22[['Soff', 'Sold to Party', 'Period', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm', 'FY']].fillna('(en blanco)')

In [19]:
df_fy22_grouped = df_fy22.groupby(['Soff', 'Sold to Party', 'Period', 'Prctr', 'WBS Element', 'PCK /FGK', 'Matnr Sitm', 'FY'], as_index=False) \
    .agg({'Total New Ord/GG1101': 'sum', 'Revenue/GG1201': 'sum', 'Sales Margin/GG1401': 'sum', 'REV Qty': 'sum', 'Tipo': 'first'})
df_fy22_grouped.shape

(156072, 13)

Clasificar Tipo de cada documento de venta

In [20]:
# Replace values of 'Tipo' with 'Canales' in df_fy22_grouped where column 'PCK /FGK' in df_pcks_p4_list
df_fy22_grouped.loc[df_fy22_grouped['PCK /FGK'].isin(df_pcks_p4['GBK/PCK']), 'Tipo'] = 'Canales'

In [21]:
# Replace values of 'Tipo' with 'SIGRAMA' in df_fy22_grouped where column 'Sold to Party' = '40004739'
df_fy22_grouped.loc[df_fy22_grouped['Sold to Party'] == '40004739', 'Tipo'] = 'SIGRAMA'
df_fy22_grouped.loc[df_fy22_grouped['Sold to Party'] == '41003426', 'Tipo'] = 'AMESA'
df_fy22_grouped.loc[df_fy22_grouped['Soff'].isin(['0', 'M001']), 'Tipo'] = 'ICB/IDB'
df_fy22_grouped.loc[df_fy22_grouped['WBS Element'] != '0', 'Tipo'] = 'Proyectos'
df_fy22_grouped.shape

(156072, 13)

## Join data from bases and Data

Se concatena la data de todos los FY en uno solo

In [22]:
# Concatenate df_fy20_grouped, df_fy21_grouped and df_fy22_grouped
df_all = pd.concat([df_fy20_grouped, df_fy21_grouped, df_fy22_grouped])
df_all.shape

(809258, 13)

Read data from bases and prepare it

In [23]:
# Read data from 'bases.xlsx' document
businessUnits = pd.read_excel('../bases/bases.xlsx', sheet_name='business_unit', usecols=['profitCenter', 'businessUnit'], dtype={'profitCenter': 'str', 'businessUnit': 'str'})
clients = pd.read_excel('../bases/bases.xlsx', sheet_name='cliente', usecols=['noCliente', 'grupo'], dtype={'noCliente': 'str', 'grupo': 'str'})
region = pd.read_excel('../bases/bases.xlsx', sheet_name='region', usecols=['clave', 'region', 'estado', 'Latitud', 'Longitud'], dtype={'clave': 'str', 'region': 'str', 'estado': 'str', 'Latitud': 'float64', 'Longitud': 'float64'})
products = pd.read_excel('../bases/bases.xlsx', sheet_name='product', usecols=['pck', 'ingeniero', 'familia', 'clasificacion', 'vsp plan', 'ebit plan'], dtype={'pck': 'str', 'ingeniero': 'str', 'familia': 'str', 'clasificacion': 'str', 'vsp plan': 'float64', 'ebit plan': 'float64'})
quarter = pd.read_excel('../bases/bases.xlsx', sheet_name='quarter', usecols=['period', 'quarter'], dtype={'period': 'str', 'quarter': 'str'})


In [24]:
# Remove duplicates from clients dataframe considering 'noCliente' column
clients = clients.drop_duplicates(subset='noCliente', keep='first')

Merge bases and Data

In [25]:
# Merge df_all with businessUnits, clients, region, products and quarter
df_all_merged = df_all.merge(businessUnits, how='left', left_on='Prctr', right_on='profitCenter').drop(['profitCenter', 'Prctr'], axis=1)
df_all_merged = df_all_merged.merge(clients, how='left', left_on='Sold to Party', right_on='noCliente').drop(['noCliente'], axis=1)
df_all_merged = df_all_merged.merge(region, how='left', left_on='Soff', right_on='clave').drop(['clave', 'Soff'], axis=1)
df_all_merged = df_all_merged.merge(products, how='left', left_on='PCK /FGK', right_on='pck').drop('pck', axis=1)
df_all_merged = df_all_merged.merge(quarter, how='left', left_on='Period', right_on='period').drop('period', axis=1)

# Rename columns of df_all_merged, 'Total New Ord/GG1101' to 'Total New Ord', 'Revenue/GG1201' to 'Revenue', 'Sales Margin/GG1401' to 'Sales Margin' and 'PCK /FGK' to 'PCK'
df_all_merged = df_all_merged.rename(columns={'Total New Ord/GG1101': 'Total New Ord', 'Revenue/GG1201': 'Revenue', 'Sales Margin/GG1401': 'Sales Margin', 'PCK /FGK': 'PCK'})
df_all_merged.columns

Index(['Sold to Party', 'Period', 'WBS Element', 'PCK', 'Matnr Sitm', 'FY',
       'Total New Ord', 'Revenue', 'Sales Margin', 'REV Qty', 'Tipo',
       'businessUnit', 'grupo', 'region', 'estado', 'Latitud', 'Longitud',
       'ingeniero', 'familia', 'clasificacion', 'vsp plan', 'ebit plan',
       'quarter'],
      dtype='object')

## Forecast Creation
FY 22 - Preparar información general para poder juntarla con el forecast de este año

In [26]:
# Create a df for forecast data of FY22 and 
df_fy22_forecast = df_all_merged.loc[(df_all_merged['FY'] == 'FY22') & (df_all_merged['Tipo'] == 'Canales')]

# Fill empty values with '(en blanco)' in df_fy22_forecast columns 'Period', 'FY', 'businessUnit', 'grupo', 'region', 'estado'
df_fy22_forecast.loc[df_fy22_forecast['Period'].isna() ,'Period'] = '(en blanco)'
df_fy22_forecast.loc[df_fy22_forecast['FY'].isna() ,'FY'] = '(en blanco)'
df_fy22_forecast.loc[df_fy22_forecast['businessUnit'].isna() ,'businessUnit'] = '(en blanco)'
df_fy22_forecast.loc[df_fy22_forecast['grupo'].isna() ,'grupo'] = '(en blanco)'
df_fy22_forecast.loc[df_fy22_forecast['region'].isna() ,'region'] = '(en blanco)'

In [27]:
# Group information of df_fy22_forecast by 'Period', 'FY', 'businessUnit', 'grupo' and 'region'
df_fy22_forecast_grouped = df_fy22_forecast.groupby(['Period', 'FY', 'businessUnit', 'grupo', 'region'], as_index=False) \
    .agg({'Total New Ord': 'sum', 'Revenue': 'sum', 'Latitud': 'first', 'Longitud': 'first', 'Sold to Party': 'first', 'quarter': 'first'})
df_fy22_forecast_grouped.shape

(2405, 11)

In [28]:
# Read data from 'bases.xlsx' document, sheet 'forecast_oficinas'
oficinas_columns = ['grupo', 'region', 'businessUnit', 'estado', 'P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12']
oficinas_types = {'grupo': 'str', 'region': 'str', 'businessUnit': 'str', 'estado': 'str', 'P01': 'float64', 'P02': 'float64', 'P03': 'float64', 'P04': 'float64', 'P05': 'float64', 'P06': 'float64', 'P07': 'float64', 'P08': 'float64', 'P09': 'float64', 'P10': 'float64', 'P11': 'float64', 'P12': 'float64'}
df_oficinas = pd.read_excel('../bases/bases.xlsx', sheet_name='forecast_oficinas', usecols=oficinas_columns, dtype=oficinas_types)

# Unpivot df_oficinas
df_oficinas = df_oficinas.melt(id_vars=['grupo', 'region', 'businessUnit', 'estado'], var_name='Period', value_name='Forecast')
df_oficinas.shape

(2448, 6)

In [29]:
# Join df_fy22_forecast_grouped and df_oficinas_unpivoted using join method
df_fy22_forecast_data = df_fy22_forecast_grouped.join(df_oficinas.set_index(['grupo', 'region', 'businessUnit', 'Period']), on=['grupo', 'region', 'businessUnit', 'Period'], how='outer')
df_fy22_forecast_data['FY'] = 'FY22'
df_fy22_forecast_data.shape

(3637, 13)

FY21 - Preparar data con el mismo procedimiento. (FY21 sí tiene split de información en estados, así que debe incluirse)

In [30]:
# Create a df for forecast data of FY21 and 
df_fy21_forecast = df_all_merged.loc[(df_all_merged['FY'] == 'FY21') & (df_all_merged['Tipo'] == 'Canales')]

# Fill empty values with '(en blanco)' in df_fy21_forecast columns 'Period', 'FY', 'businessUnit', 'grupo', 'region', 'estado'
df_fy21_forecast.loc[df_fy21_forecast['Period'].isna() ,'Period'] = '(en blanco)'
df_fy21_forecast.loc[df_fy21_forecast['FY'].isna() ,'FY'] = '(en blanco)'
df_fy21_forecast.loc[df_fy21_forecast['businessUnit'].isna() ,'businessUnit'] = '(en blanco)'
df_fy21_forecast.loc[df_fy21_forecast['grupo'].isna() ,'grupo'] = '(en blanco)'
df_fy21_forecast.loc[df_fy21_forecast['region'].isna() ,'region'] = '(en blanco)'
df_fy21_forecast.loc[df_fy21_forecast['estado'].isna() ,'estado'] = '(en blanco)'

# df_fy21_forecast[['Period', 'FY', 'businessUnit', 'grupo', 'region', 'estado']] = df_fy21_forecast[['Period', 'FY', 'businessUnit', 'grupo', 'region', 'estado']].fillna('(en blanco)')

df_fy21_forecast.shape

(317590, 23)

In [31]:
# Group information of df_fy21_forecast by 'Period', 'FY', 'businessUnit', 'grupo' and 'region'
df_fy21_forecast_grouped = df_fy21_forecast.groupby(['Period', 'FY', 'businessUnit', 'grupo', 'region', 'estado'], as_index=False) \
    .agg({'Total New Ord': 'sum', 'Revenue': 'sum', 'Latitud': 'first', 'Longitud': 'first', 'Sold to Party': 'first', 'quarter': 'first'})
df_fy21_forecast_grouped.shape

(5042, 12)

In [32]:
# Read data from 'bases.xlsx' document, sheet 'forecast_fy21'
oficinas_columns = ['grupo', 'region', 'businessUnit', 'estado', 'P01', 'P02', 'P03', 'P04', 'P05', 'P06', 'P07', 'P08', 'P09', 'P10', 'P11', 'P12']
oficinas_types = {'grupo': 'str', 'region': 'str', 'businessUnit': 'str', 'estado': 'str', 'P01': 'float64', 'P02': 'float64', 'P03': 'float64', 'P04': 'float64', 'P05': 'float64', 'P06': 'float64', 'P07': 'float64', 'P08': 'float64', 'P09': 'float64', 'P10': 'float64', 'P11': 'float64', 'P12': 'float64'}
df_oficinas = pd.read_excel('../bases/bases.xlsx', sheet_name='forecast_fy21', usecols=oficinas_columns, dtype=oficinas_types)

# Unpivot df_oficinas
df_oficinas = df_oficinas.melt(id_vars=['grupo', 'region', 'businessUnit', 'estado'], var_name='Period', value_name='Forecast')
df_oficinas.shape

(44820, 6)

In [33]:
# Join df_fy21_forecast_grouped and df_oficinas using join method
df_fy21_forecast_data = df_fy21_forecast_grouped.join(df_oficinas.set_index(['grupo', 'region', 'businessUnit', 'Period', 'estado']), on=['grupo', 'region', 'businessUnit', 'Period', 'estado'], how='outer')
df_fy21_forecast_data['FY'] = 'FY21'
df_fy21_forecast_data.shape

(46980, 13)

## Concatenate data of Fy21 and FY22

In [34]:
# concat df_fy22_forecast_data and df_fy21_forecast_data
df_forecast_data = pd.concat([df_fy22_forecast_data, df_fy21_forecast_data])
df_forecast_data.shape

(50617, 13)

In [35]:
# Read data from "categoria_descuento" sheet of "bases.xlsx" document
categoria_descuento_columns = ['FY', 'grupo', 'businessUnit', 'tipo_cliente', 'descuento_cliente', 'beneficio']
categoria_descuento_types = {'FY': 'str', 'grupo': 'str', 'businessUnit': 'str', 'tipo_cliente': 'str', 'descuento_cliente': 'float64', 'beneficio': 'float64'}
df_categoria_descuento = pd.read_excel('../bases/bases.xlsx', sheet_name='categoria_descuento', usecols=categoria_descuento_columns, dtype=categoria_descuento_types)

df_categoria_descuento.shape


(552, 6)

In [36]:
# join df_forecast_data and df_categoria_descuento using merge method
df_forecast_categorias_data = df_forecast_data.merge(df_categoria_descuento, on=['FY', 'grupo', 'businessUnit'], how='left')

# Rename columns of df_forecast_categorias_data
forecast_column_names = ['Periodo', 'FY', 'BU', 'GRUPO', 'Región', 'Total New Ord', 'Revenue', 'Latitud', 'Longitud', 'Sold to Party', 'quarter', 'estado', 'Forecast', 'tipo_cliente', 'descuento_cliente', 'Beneficio']
df_forecast_categorias_data.columns = forecast_column_names
df_forecast_categorias_data.shape

(50617, 16)

In [37]:
# export to csv 'df_forecast_categorias_data.csv' format utf-8
df_forecast_categorias_data.to_csv('../out/forecast_data2.csv', index=False, encoding='utf-8')

# Marketing points

In [38]:
# Fill empty values with '(en blanco)' in df_forecast_categorias_data columns 'quarter', 'GRUPO', 'BU', 'FY'
df_forecast_categorias_data.loc[df_forecast_categorias_data['quarter'].isna() ,'quarter'] = '(en blanco)'
df_forecast_categorias_data.loc[df_forecast_categorias_data['GRUPO'].isna() ,'GRUPO'] = '(en blanco)'
df_forecast_categorias_data.loc[df_forecast_categorias_data['BU'].isna() ,'BU'] = '(en blanco)'
df_forecast_categorias_data.loc[df_forecast_categorias_data['FY'].isna() ,'FY'] = '(en blanco)'

In [39]:
# Group by 'quarter', 'grupo' and 'BU'
df_forecast_categorias_data_grouped = df_forecast_categorias_data.groupby(['quarter', 'GRUPO', 'BU', 'FY'], as_index=False) \
    .agg({'Total New Ord': 'sum', 'Forecast': 'sum', 'tipo_cliente': 'first',  'Beneficio': 'first'})

# filter df_forecast_categorias_data_grouped by 'forecast' = 0
df_forecast_categorias_data_grouped = df_forecast_categorias_data_grouped[df_forecast_categorias_data_grouped['Forecast'] != 0]
df_forecast_categorias_data_grouped.shape

(1359, 8)

In [40]:
# New column 'creditos' = 'Total New Ord' * 'beneficio' filtering where 'Total New Ord' > (forecast / 2)
validation_NO_fC = df_forecast_categorias_data_grouped['Total New Ord'] >= (df_forecast_categorias_data_grouped['Forecast'] / 2)
df_forecast_categorias_data_grouped['creditos'] = df_forecast_categorias_data_grouped[validation_NO_fC]['Total New Ord'] * df_forecast_categorias_data_grouped[validation_NO_fC]['Beneficio']


Date validation

In [41]:
month_quarter = {1: 'Q2', 2: 'Q2', 3: 'Q2', 4: 'Q3', 5: 'Q3', 6: 'Q3', 7: 'Q4', 8: 'Q4', 9: 'Q4', 10: 'Q1', 11: 'Q1', 12: 'Q1'}
quarters = ['Q1', 'Q2', 'Q3', 'Q4']

# get today's month
today = datetime.date.today()
month = today.month

In [42]:
# find previous quarters of today's month
validation_quarter = []
for quarter in quarters:
    if month_quarter[month] != quarter:
        validation_quarter.append(quarter)
    else:
        break
# if quarter is Q4, then add Q4 to validation_quarter
if month_quarter[month] == 'Q4':
    validation_quarter.append('Q4')

validation_quarter


['Q1', 'Q2']

In [43]:
# Set 'creditos' = 0 for all quarters except 'validation_quarter'
df_forecast_categorias_data_grouped.loc[-df_forecast_categorias_data_grouped['quarter'].isin(validation_quarter), 'creditos'] = 0

In [44]:
df_forecast_categorias_data_grouped.to_csv('../out/testing.csv', index=False, encoding='utf-8')

In [45]:
# df_fy21_forecast_data.to_csv('../testing.csv', index=False)

In [46]:
# df_fy22.to_csv('../fy22.csv', index=False)